In [2]:
import pandas as pd

Form a graph with people as nodes with edges between them if they have starred in the same movie. Search for some correlation between how well connected people are and the kind of movies they have performed in / the type of role they have etc.
Once such hypotheses have been formed, test them in a similar graph in the imdb movie dataset to see if they hold there as well. 

Test the hypothesis that an uncommon combination of genres means a somewhat extreme box office collection - either low or high



Is benford's law scale invariant?

In [22]:
movie_df = pd.read_csv('data/movie.metadata.tsv', sep='\t', names=['wiki_movie_id', 'freebase_movie_id', 'title', 'release_date', 'box_office_revenue', 'runtime', 'languages', 'countries', 'genres'], index_col='freebase_movie_id')
movie_df = movie_df.reset_index()

In [ ]:
# No duplicates of freebase_movie_id
movie_df.groupby('freebase_movie_id').agg(count = ('freebase_movie_id', 'size')).sort_values('count', ascending=False)

,count
freebase_movie_id,
/m/011_mj,1
/m/0bwg_bz,1
/m/0bwgk61,1
/m/0bwgjgq,1
/m/0bwghsn,1
...,...
/m/04n51dq,1
/m/04n5187,1
/m/04n514t,1


In [24]:
col_names = ["wiki_movie_id", "freebase_movie_id", "movie_release_date", "character_name", "actor_date_of_birth", "Actor gender", "Actor height (in meters)", "Actor ethnicity (Freebase ID)", "Actor name", "Actor age at movie release", "Freebase character/actor map ID", "Freebase character ID", "Freebase actor ID"]
character_df = pd.read_csv('data/character.metadata.tsv', sep='\t', names=col_names, index_col="Freebase character ID")
character_df.columns

Index(['wiki_movie_id', 'freebase_movie_id', 'movie_release_date',
       'character_name', 'actor_date_of_birth', 'Actor gender',
       'Actor height (in meters)', 'Actor ethnicity (Freebase ID)',
       'Actor name', 'Actor age at movie release',
       'Freebase character/actor map ID', 'Freebase actor ID'],
      dtype='object')

In [ ]:
character_df[]

In [7]:
movie_character_df = movie_df.merge(character_df, on='freebase_movie_id', how='inner')
movie_character_df.columns

Index(['freebase_movie_id', 'wiki_movie_id_x', 'title', 'release_date',
       'box_office_revenue', 'runtime', 'languages', 'countries', 'genres',
       'wiki_movie_id_y', 'movie_release_date', 'character_name',
       'actor_date_of_birth', 'Actor gender', 'Actor height (in meters)',
       'Actor ethnicity (Freebase ID)', 'Actor name',
       'Actor age at movie release', 'Freebase character/actor map ID',
       'Freebase actor ID'],
      dtype='object')

In [ ]:
# Investigating if Actor name and title uniquely identifies a row.
# Realizing maybe duplicate rows!
temp = movie_character_df.groupby(['Actor name', 'title', 'character_name']).agg(count = ('Actor name', 'size'))
temp.sort_values('count', ascending=False)

,,,count
Actor name,title,character_name,
Olivia Thirlby,The Other Side,Max Mackenzie,3
Rose McGowan,Dead Awake,Charlie,3
Aryeman Ramsay,Good Luck!,Vicky Varma,3
Ayub Khan,The Death Sentence,Vinay Singh,3
Aryeman Ramsay,Ek Aadat,Armaan,3
...,...,...,...
Gy Mirano,Call Me,Waitress,1
Gyalsen Gurung,Himalaya,Chewan,1
Gyanesh Mukherjee,Target,Rampear,1


In [ ]:
# Duplicates in the dataset
movie_character_df[
    (movie_character_df['Actor name'] == 'Olivia Thirlby') & 
    (movie_character_df['title'] == 'The Other Side') & 
    (movie_character_df['character_name']== 'Max Mackenzie')]

,freebase_movie_id,wiki_movie_id_x,title,release_date,box_office_revenue,runtime,languages,countries,genres,wiki_movie_id_y,movie_release_date,character_name,actor_date_of_birth,Actor gender,Actor height (in meters),Actor ethnicity (Freebase ID),Actor name,Actor age at movie release,Freebase character/actor map ID,Freebase actor ID
30260,/m/09gknmp,24904539,The Other Side,2012,NaN,NaN,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America"", ""/m/...","{""/m/07s9rl0"": ""Drama"", ""/m/02n4kr"": ""Mystery""...",24904539,2012,Max Mackenzie,1986-10-06,F,1.626,NaN,Olivia Thirlby,25.0,/m/0h5qn1c,/m/025yjfc
30271,/m/09gknmp,24904539,The Other Side,2012,NaN,NaN,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America"", ""/m/...","{""/m/07s9rl0"": ""Drama"", ""/m/02n4kr"": ""Mystery""...",24904539,2012,Max Mackenzie,1986-10-06,F,1.626,NaN,Olivia Thirlby,25.0,/m/0chbfb2,/m/025yjfc
30273,/m/09gknmp,24904539,The Other Side,2012,NaN,NaN,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America"", ""/m/...","{""/m/07s9rl0"": ""Drama"", ""/m/02n4kr"": ""Mystery""...",24904539,2012,Max Mackenzie,1986-10-06,F,1.626,NaN,Olivia Thirlby,25.0,/m/0h38m4x,/m/025yjfc


In [18]:
movie_character_df[
    (movie_character_df['Actor name'] == 'Olivia Thirlby') & 
    (movie_character_df['title'] == 'The Other Side') & 
    (movie_character_df['character_name']== 'Max Mackenzie')]['countries']

30260    {"/m/09c7w0": "United States of America", "/m/...
30271    {"/m/09c7w0": "United States of America", "/m/...
30273    {"/m/09c7w0": "United States of America", "/m/...
Name: countries, dtype: object

In [56]:
oscar_df = pd.read_csv('data/the_oscar_award.csv')
oscar_df = oscar_df[oscar_df['film'].notna() & oscar_df['name'].notna()]

In [57]:
# Filtering relevant Oscar catagories
# TODO: explain why we drop certain categories.
oscar_df = oscar_df[oscar_df['category'].str.contains('ACTOR') | oscar_df['category'].str.contains('ACTRESS')]

This join needs to be checked. 
Joining with ignore case? 
Do we want to do a right join? (so also keep movies that did not win oscars)


In [58]:
# Avoid case-sensitivity and extra spaces
oscar_df['film_title_parsed'] = oscar_df['film'].str.lower().str.strip()
oscar_df['name_parsed'] = oscar_df['name'].str.lower().str.strip()

movie_character_df['film_title_parsed'] = movie_character_df['title'].str.lower().str.strip()
movie_character_df['name_parsed'] = movie_character_df['Actor name'].str.lower().str.strip()

In [59]:
oscar_character_movie_df = movie_character_df.merge(oscar_df, on=['film_title_parsed', 'name_parsed'], how='inner')